In [1]:
from mysql.connector import connect
import pygsheets
import pandas as pd
import datetime

In [2]:
connection = connect(host='localhost', database='hcmopsinventory', user='root', password='BaeminInventory@123')
if connection.is_connected():
    db_Info = connection.get_server_info()
    cursor = connection.cursor(buffered=True)
else:
    print('Not connected.')

In [10]:
# cursor.execute('DELETE FROM outboundscanlog')
# connection.commit()

In [22]:
# connection.close()

In [3]:
# authorization to gsheet
gc = pygsheets.authorize(service_file='C:/Users/Hung Vu Tien/Documents/Baemin/Data/Inventory/mySQL_connector/inventory/credentialKeyJson/inventorywithpythonandmysql-1e9415153787.json')

In [4]:
# # Create empty dataframe
# df = pd.DataFrame()

# # Create a column
# df['name'] = ['Hung', 'Steve', 'Sarah']

# #open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
# sh = gc.open('Test connect by python')

# #select the first sheet 
# wks = sh[0]

# #update the first sheet with df, starting at cell B2. 
# wks.set_dataframe(df,(1,1))

# # add a sheet with 20 rows and 2 columns
# sh.add_worksheet(rows=20,cols=2,title='runs')

In [4]:
# Open spreadsheet inbound 
sh_inbound = gc.open('HCMOPS_INVENTORY_MANAGEMENT_INBOUND')

# Open spreadsheet outbound
sh_outbound = gc.open('HCMOPS_INVENTORY_MANAGEMENT_OUTBOUND')

# Open spreadsheet master
sh_barcode = gc.open('HCMOPS_INVENTORY_MANAGEMENT')
barcode = sh_barcode.worksheets('title','PRINT_BARCODE')[0]

In [5]:
# Get data in spreadsheet Inbound scan, sheet("InboundScan")
ibScan = sh_inbound[1]

# Get data in spreadsheet Outbound scan, sheet("OutboundScanLog_VM")
obVMScan = sh_outbound.worksheets('title','OutboundScanLog_VM')[0]

# Get data in spreadsheet Outbound scan, sheet("OutboundScanLog")
obScan = sh_outbound.worksheets('title','OutboundScanLog')[0]

In [6]:
# get all the records of inbound
records_data_ib = ibScan.get_all_records()

# get all the records of outbound
records_data_ob = obScan.get_all_records()

# get all the records of outboundvm
records_data_obvm = obVMScan.get_all_records()

In [7]:
# Get data in spreadsheet master, sheet("PRINT_BARCODE")
for i in range(0, len(barcode.get_all_records(head=3))):
    if barcode.get_all_records(head=3)[i]['Ngày nhập'] == '':
        records_data_barcode = barcode.get_all_records(head=3)[0:i]
        break

In [27]:
# val_ob_to_sql = [(i['Scan_Field'], datetime.datetime.strptime(i['Scan_Date'], '%m/%d/%Y %H:%M:%S')) for i in records_data_ob]
# # Write into database
# query = 'INSERT INTO outboundscanlog VALUES (%s, %s)'
# cursor.executemany(query, val_ob_to_sql)
# connection.commit()

In [8]:
# get barcode database
cursor.execute('select * from barcodehcm')
dbBarcode = cursor.fetchall()

# Barcode scan insert into database
if len(records_data_barcode) > len(dbBarcode):
    records_data_barcode_write = records_data_barcode[len(dbBarcode):]
    val_barcode_to_sql = [
        (
            datetime.datetime.strptime(i['Ngày nhập'], '%m/%d/%Y'),
            i['Loại'],
            i['Loại_Encode'],
            i['Số lượng'],
            i['Encode'],
            i['Barcode'].replace('*', '')
        ) 
        for i in records_data_barcode_write
    ]
    query = 'INSERT INTO barcodehcm VALUES (%s, %s, %s, %s, %s, %s)'
    cursor.executemany(query, val_barcode_to_sql)
    connection.commit()
else:
    print('Not import new data Barcode.')

Not import new data Barcode.


In [9]:
val_barcode_to_sql

NameError: name 'val_barcode_to_sql' is not defined

In [10]:
# get inbound database
cursor.execute('select * from inboundscanlog')
dbInboundScan = cursor.fetchall()

# Inbound scan insert into database
if len(records_data_ib) > len(dbInboundScan):
    records_data_ib_write = records_data_ib[len(dbInboundScan):]
    val_ib_to_sql = [
        (
            i['Scan_Field'], 
            datetime.datetime.strptime(i['Scan_Date'], '%m/%d/%Y %H:%M:%S')
        ) 
        for i in records_data_ib_write
    ]
    query = 'INSERT INTO inboundscanlog VALUES (%s, %s)'
    cursor.executemany(query, val_ib_to_sql)
    connection.commit()
else:
    print('Not import new data Inboundscanlog.')

Not import new data Inboundscanlog.


In [11]:
# get outbound database
cursor.execute('select * from outboundscanlog')
dbOutboundscan = cursor.fetchall()

# Outbound scan insert into database
if len(records_data_ob) > len(dbOutboundscan):
    records_data_ob_write = records_data_ob[len(dbOutboundscan):]
    val_ob_to_sql = [
        (
            i['Scan_Field'], 
            datetime.datetime.strptime(i['Scan_Date'], '%m/%d/%Y %H:%M:%S')
        ) 
        for i in records_data_ob_write
    ]
    query = 'INSERT INTO outboundscanlog VALUES (%s, %s)'
    cursor.executemany(query, val_ob_to_sql)
    connection.commit()
else:
    print('Not import new data Outboundscanlog.')

In [12]:
# get outboundvm database
cursor.execute('select * from outboundscanlogvm')
dbOutboundscanVM = cursor.fetchall()

# Outbound scan VM insert into database
if len(records_data_obvm) > len(dbOutboundscanVM):
    records_data_obvm_write = records_data_obvm[len(dbOutboundscanVM):]
    val_obvm_to_sql = [
        (
            i['Scan_Field'], 
            datetime.datetime.strptime(i['Scan_Date'], '%m/%d/%Y %H:%M:%S')
        ) 
        for i in records_data_obvm_write
    ]
    query = 'INSERT INTO outboundscanlogvm VALUES (%s, %s)'
    cursor.executemany(query, val_obvm_to_sql)
    connection.commit()
else:
    print('Not import new data OutboundscanlogVM')

In [13]:
connection.close()

In [10]:
len(records_data_obvm)

325

In [11]:
len(dbOutboundscanVM)

307

In [13]:
len(records_data_ob)

1160

In [14]:
len(dbOutboundscan)

1135

In [16]:
len(records_data_barcode)

338

In [17]:
len(dbBarcode)

337

In [19]:
len(records_data_ib)

338

In [20]:
len(dbInboundScan)

337

In [75]:
# cursor.execute('DELETE FROM inboundscanlog')
# connection.commit()

In [22]:
cursor.execute('select * from outboundscanlog')
dbOutboundscan = cursor.fetchall()

In [23]:
dbOutboundscan

[('21111900TS000L1I', datetime.datetime(2021, 11, 22, 15, 55)),
 ('21111900TS03XL1I', datetime.datetime(2021, 11, 22, 15, 55)),
 ('2111190NKK00XL4I', datetime.datetime(2021, 11, 22, 15, 57)),
 ('2111190NKK00XL4I', datetime.datetime(2021, 11, 22, 15, 58)),
 ('21111900JK000L2I', datetime.datetime(2021, 11, 22, 15, 59)),
 ('21111900JK03XL1I', datetime.datetime(2021, 11, 22, 15, 59)),
 ('2111190TUI00001I', datetime.datetime(2021, 11, 22, 16, 0)),
 ('2111190TUI00001I', datetime.datetime(2021, 11, 22, 16, 0)),
 ('21111900TS000L1I', datetime.datetime(2021, 11, 22, 16, 0)),
 ('21111900TS03XL1I', datetime.datetime(2021, 11, 22, 15, 55)),
 ('21111900JK000L2I', datetime.datetime(2021, 11, 22, 15, 59)),
 ('21111900JK03XL1I', datetime.datetime(2021, 11, 22, 15, 59)),
 ('2111190TUI00001I', datetime.datetime(2021, 11, 22, 16, 0)),
 ('2111190TUI00001I', datetime.datetime(2021, 11, 22, 16, 0)),
 ('21111900AM002P1I', datetime.datetime(2021, 11, 22, 16, 3)),
 ('21111900JK000M1I', datetime.datetime(2021, 